<a href="https://colab.research.google.com/github/yashvirsingh05/AddingValues/blob/master/Sentiment_Analysis_Amazon_product_review.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Importing the datasets**


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import spacy


In [3]:
from google.colab import files
upload = files.upload()

Saving yelp_labelled.txt to yelp_labelled.txt


In [4]:
df=pd.read_csv('yelp_labelled.txt',sep='\t',header=None)
df.head()

,0,1
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [5]:
cname=['Review','Sentiment']
df.columns=cname

In [6]:
df.head()
#0-Negative review
#1-positive review

,Review,Sentiment
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [7]:
df.shape

(1000, 2)

# Add another dataset

In [8]:
from google.colab import files
upload=files.upload()

Saving amazon_cells_labelled.txt to amazon_cells_labelled.txt


In [9]:
data_amazon=pd.read_csv('amazon_cells_labelled.txt',sep='\t',header=None)
data_amazon.head()

,0,1
0,So there is no way for me to plug it in here i...,0
1,"Good case, Excellent value.",1
2,Great for the jawbone.,1
3,Tied to charger for conversations lasting more...,0
4,The mic is great.,1


In [11]:
cname=['Review','Sentiment']
data_amazon.columns=cname
data_amazon.head()

,Review,Sentiment
0,So there is no way for me to plug it in here i...,0
1,"Good case, Excellent value.",1
2,Great for the jawbone.,1
3,Tied to charger for conversations lasting more...,0
4,The mic is great.,1


In [12]:
data_amazon.shape

(1000, 2)

In [13]:
from google.colab import files
upload=files.upload()

Saving imdb_labelled.txt to imdb_labelled.txt


In [15]:
data_imdb=pd.read_csv('imdb_labelled.txt',sep='\t',header=None)
data_imdb.head()

,0,1
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0
3,Very little music or anything to speak of.,0
4,The best scene in the movie was when Gerardo i...,1


In [16]:
cname=['Review','Sentiment']
data_imdb.columns=cname
data_imdb.head()

,Review,Sentiment
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0
3,Very little music or anything to speak of.,0
4,The best scene in the movie was when Gerardo i...,1


In [18]:
data_imdb.shape

(748, 2)

# add all data in a single dataframe

In [19]:
data=df.append([data_amazon,data_imdb],ignore_index=True)
data.shape

(2748, 2)

In [20]:
data['Sentiment'].value_counts()
#1-positive reviews
#0-negative reviews

1    1386
0    1362
Name: Sentiment, dtype: int64

In [23]:
#check for null values
data.isnull().sum()

Review       0
Sentiment    0
dtype: int64

In [42]:
x=data['Review']
y=data['Sentiment']

In [43]:
x.shape

(2748,)

# Data cleaning

In [25]:
import string
nlp=spacy.load('en_core_web_sm')
punc = string.punctuation
punc

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [26]:
from spacy.lang.en.stop_words import STOP_WORDS
stopwords = list(STOP_WORDS)


In [27]:
def data_cleaning(sentence):
  doc = nlp(sentence)
  tokens = [] 
  for token in doc:
    if token.lemma_ != "-PRON-":
      temp = token.lemma_.lower().strip()
    else:
      temp = token.lower_
    tokens.append(temp)
 
  cleaned_tokens = []
  for token in tokens:
    if token not in stopwords and token not in punc:
      cleaned_tokens.append(token)
  return cleaned_tokens

In [28]:
data_cleaning("Hello all, It's a beautiful day outside there!")

['hello', 'beautiful', 'day', 'outside']

# Vectorization Feature Engineering




In [31]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline

In [32]:
tfidf = TfidfVectorizer(tokenizer=data_cleaning)

# Train and Test the model

In [51]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 1)

In [52]:
x_train.head()

579     As for the service: I'm a fan, because it's qu...
1178    It only recognizes the Phone as its storage de...
829     I do love sushi, but I found Kabuki to be over...
452     I am far from a sushi connoisseur but I can de...
1870                                          Works fine.
Name: Review, dtype: object

In [53]:
x_test.head()

1112    its a little geeky but i think thats its sex o...
1923                                          cool phone.
1301              Now I know that I made a wise decision.
1442    They made this case too small and is very diff...
428                     Strike 2, who wants to be rushed.
Name: Review, dtype: object

# Support Vector Machine

In [54]:
from sklearn.svm import LinearSVC
classifier = LinearSVC()

In [55]:
clf = Pipeline([('tfidf',tfidf), ('clf',classifier)])
clf.fit(x_train, y_train)

Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function data_cleaning at 0x7f20b276ea70>,
                                 use_idf=True, vocabulary=None)),
                ('clf',
                 LinearSVC(C=1.0, class_weight=None, dual=True,
                           fit_intercept=True

In [56]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
y_pred = clf.predict(x_test)
confusion_matrix(y_test, y_pred)

array([[228,  50],
       [ 52, 220]])

In [57]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.81      0.82      0.82       278
           1       0.81      0.81      0.81       272

    accuracy                           0.81       550
   macro avg       0.81      0.81      0.81       550
weighted avg       0.81      0.81      0.81       550



In [61]:
accuracy_score(y_test,y_pred)

0.8145454545454546

In [62]:
clf.predict(["Wow, I am learning Natural Language Processing in fun fashion!"])

array([1])

# Logistic Regression


In [63]:
from sklearn.linear_model import LogisticRegression

In [64]:
classi = LogisticRegression()

In [65]:
cl = Pipeline([('fidf',tfidf), ('cl',classi)])
cl.fit(x_train, y_train)

Pipeline(memory=None,
         steps=[('fidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(...\w\\w+\\b',
                                 tokenizer=<function data_cleaning at 0x7f20b276ea70>,
                                 use_idf=True, vocabulary=None)),
                ('cl',
                 LogisticRegression(C=1.0, class_weight=None, dual=False,
                                    fit_i

In [66]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
y_pred = cl.predict(x_test)
confusion_matrix(y_test, y_pred)

array([[227,  51],
       [ 49, 223]])

In [67]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.82      0.82      0.82       278
           1       0.81      0.82      0.82       272

    accuracy                           0.82       550
   macro avg       0.82      0.82      0.82       550
weighted avg       0.82      0.82      0.82       550



In [68]:
accuracy_score(y_test, y_pred)

0.8181818181818182

# Decision Tree

In [69]:
from sklearn.tree import DecisionTreeClassifier

In [70]:
cla = DecisionTreeClassifier()
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 1)

In [71]:
cll = Pipeline([('tfidf',tfidf), ('cla',cla)])
cll.fit(x_train, y_train)

Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='...
                                 use_idf=True, vocabulary=None)),
                ('cla',
                 DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                        criterion='gini', max_depth=None,
                                        max_features=None, max_l

In [72]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
y_pred = cll.predict(x_test)
confusion_matrix(y_test, y_pred)

array([[215,  63],
       [ 67, 205]])

In [73]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.76      0.77      0.77       278
           1       0.76      0.75      0.76       272

    accuracy                           0.76       550
   macro avg       0.76      0.76      0.76       550
weighted avg       0.76      0.76      0.76       550



In [74]:
accuracy_score(y_test, y_pred)

0.7636363636363637

# Random forest

In [75]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 100, random_state = 42)



In [76]:
cll = Pipeline([('tfidf',tfidf), ('cla',classifier)])
cll.fit(x_train, y_train)

Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='...
                 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                        class_weight=None, criterion='gini',
                                        max_depth=None, max_features='auto',
                                        max_leaf_nodes=None, max_samples=None

In [77]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
y_pred = cll.predict(x_test)
confusion_matrix(y_test, y_pred)

array([[236,  42],
       [ 72, 200]])

In [78]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.77      0.85      0.81       278
           1       0.83      0.74      0.78       272

    accuracy                           0.79       550
   macro avg       0.80      0.79      0.79       550
weighted avg       0.80      0.79      0.79       550



In [79]:
accuracy_score(y_test, y_pred)

0.7927272727272727